In [1]:
from hedypet.utils import DERIVATIVES_ROOT, RAW_ROOT, load_splits, load_sidecar, get_time_frames_midpoint
from nifti_dynamic.utils import load_tac
from nifti_dynamic.patlak import roi_patlak
from parse import parse

In [ ]:
subs = load_splits()["all"]

def run_save_patlak(tac,if_tac,frame_time_middle, out_dir, region_name, n_frames_regression=4,save_img=False):
    slope, intercept, X, Y = roi_patlak(tac,if_tac,frame_time_middle,n_frames_regression)
    ki_path = os.path.join(out_dir,f"{region_name}_Ki.txt")
    os.makedirs(out_dir,exist_ok=True)

    if save_img:
        plt.figure(figsize=(4,3))
        plot_patplak(X,Y,slope,intercept,n_frames_regression)
        plt.title(region_name)
        plt.ylim(np.nanmin(Y),np.nanmax(Y[-n_frames_regression:])*1.1)
        plt.legend()
        plt.savefig(ki_path.replace(".txt",".jpg"))

In [21]:
12*499

5988

In [10]:
tacs_if

[PosixPath('/depict/data/hedit/derivatives/tacs/sub-000/acdynPSF/aortasegments/erosion-0/tac_1'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-000/acdynPSF/aortasegments/erosion-0/tac_2'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-000/acdynPSF/aortasegments/erosion-0/tac_3'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-000/acdynPSF/aortasegments/erosion-0/tac_4'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-000/acdynPSF/aortasegments/erosion-1/tac_1'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-000/acdynPSF/aortasegments/erosion-1/tac_2'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-000/acdynPSF/aortasegments/erosion-1/tac_3'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-000/acdynPSF/aortasegments/erosion-1/tac_4'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-000/acdynPSF/aortavois_ml-1_width-3/erosion-0/tac_1'),
 PosixPath('/depict/data/hedit/derivatives/tacs/sub-000/acdynPSF/aortavois_ml-1_width-3/erosion-0/tac_2'),
 P

In [ ]:
from tqdm import tqdm
import warnings
import pandas as pd 

warnings.filterwarnings("ignore")
frames = [2,3,4,5,6,7,8,9,10,11,12]

ki_data = []
for sub in ["sub-000"]:
    tacs_root = (DERIVATIVES_ROOT / f"tacs/{sub}/acdynPSF")
    tacs = list(tacs_root.glob("**/tac*"))
    tacs_if = [x for x in tacs if "aorta" in str(x)]
    tacs_organs = [x for x in tacs if x not in tacs_if]
    t_middle = get_time_frames_midpoint(sub)

    # Try all combinations of inputs functions and num_frames
    for tac_if_path in tqdm(tacs_if):
        tac_if, _, _ = load_tac(tac_if_path)
        for tac_organ_path in tacs_organs:
            tac_organ, _, _ = load_tac(tac_organ_path)
            for frame in frames:
                slope, intercept, X, Y = roi_patlak(tac_organ,tac_if,t_middle,frame)
                ki_data.append({"slope":float(slope),"tac_if_path":tac_if_path,"tac_organ_path":tac_organ_path,"frames":frame})

df = pd.DataFrame(ki_data)

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:35<00:00,  2.93s/it]


In [8]:
df

,slope,tac_if_path,tac_organ_path,frames
0,0.001876,/depict/data/hedit/derivatives/tacs/sub-000/ac...,/depict/data/hedit/derivatives/tacs/sub-000/ac...,2
1,0.001907,/depict/data/hedit/derivatives/tacs/sub-000/ac...,/depict/data/hedit/derivatives/tacs/sub-000/ac...,3
2,0.001614,/depict/data/hedit/derivatives/tacs/sub-000/ac...,/depict/data/hedit/derivatives/tacs/sub-000/ac...,4
3,0.001476,/depict/data/hedit/derivatives/tacs/sub-000/ac...,/depict/data/hedit/derivatives/tacs/sub-000/ac...,5
4,0.001261,/depict/data/hedit/derivatives/tacs/sub-000/ac...,/depict/data/hedit/derivatives/tacs/sub-000/ac...,6
...,...,...,...,...
65863,0.000397,/depict/data/hedit/derivatives/tacs/sub-000/ac...,/depict/data/hedit/derivatives/tacs/sub-000/ac...,8
65864,0.000408,/depict/data/hedit/derivatives/tacs/sub-000/ac...,/depict/data/hedit/derivatives/tacs/sub-000/ac...,9
65865,0.000411,/depict/data/hedit/derivatives/tacs/sub-000/ac...,/depict/data/hedit/derivatives/tacs/sub-000/ac...,10
65866,0.000412,/depict/data/hedit/derivatives/tacs/sub-000/ac...,/depict/data/hedit/derivatives/tacs/sub-000/ac...,11
